<a href="https://colab.research.google.com/github/GuyInFreezer/neural-network-challenge-2/blob/main/attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [7]:
# Create a list of at least 10 column names to use as X data
X_features = ['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobSatisfaction', 'NumCompaniesWorked', 'PerformanceRating', 'WorkLifeBalance', 'YearsSinceLastPromotion']

# Create X_df using your selected columns
X_df = attrition_df[X_features]

# Show the data types for X_df
X_df.dtypes


Age                        int64
DistanceFromHome           int64
Education                  int64
EnvironmentSatisfaction    int64
HourlyRate                 int64
JobInvolvement             int64
JobSatisfaction            int64
NumCompaniesWorked         int64
PerformanceRating          int64
WorkLifeBalance            int64
YearsSinceLastPromotion    int64
dtype: object

In [8]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df)


In [52]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
y_df['Department'].value_counts()

Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

In [9]:
# Create a StandardScaler
sscaler = StandardScaler()

# Fit the StandardScaler to the training data
sscaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = sscaler.transform(X_train)
X_test_scaled = sscaler.transform(X_test)


In [30]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit the encoder to the training data
encoder.fit(y_train['Department'].values.reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded = encoder.transform(y_train['Department'].values.reshape(-1,1))
y_test_encoded = encoder.transform(y_test['Department'].values.reshape(-1,1))
y_train_encoded


array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [29]:
# Create a OneHotEncoder for the Attrition column
encoder2 = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit the encoder to the training data
encoder2.fit(y_train['Attrition'].values.reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded2 = encoder2.transform(y_train['Attrition'].values.reshape(-1,1))
y_test_encoded2 = encoder2.transform(y_test['Attrition'].values.reshape(-1,1))
y_train_encoded2


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [43]:
# Find the number of columns in the X training data
num_input = len(X_train.columns)

# Create the input layer
input_layer = layers.Input(shape=(X_df.shape[1]), name = 'input')

# Create at least two shared layers
shared1 = layers.Dense(64, activation = 'relu', name = 'shared1')(input_layer)
shared2 = layers.Dense(128, activation = 'relu', name = 'shared2')(shared1)

In [44]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, name = 'department_hidden')(shared2)

# Create the output layer
department_output = layers.Dense(3, activation = 'softmax', name = 'department_output')(department_hidden)


In [45]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, name = 'attrition_hidden')(shared2)

# Create the output layer
attrition_output = layers.Dense(2, activation = 'sigmoid', name = 'attrition_output')(attrition_hidden)


In [46]:
# Create the model
model = Model(inputs = input_layer, outputs = [department_output, attrition_output])

# Compile the model
model.compile(optimizer = 'adam', loss={'department_output':'categorical_crossentropy', 'attrition_output':'binary_crossentropy'},
              metrics={'department_output':'accuracy', 'attrition_output':'accuracy'})

# Summarize the model
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 11)]                 0         []                            
                                                                                                  
 shared1 (Dense)             (None, 64)                   768       ['input[0][0]']               
                                                                                                  
 shared2 (Dense)             (None, 128)                  8320      ['shared1[0][0]']             
                                                                                                  
 department_hidden (Dense)   (None, 32)                   4128      ['shared2[0][0]']             
                                                                                            

In [47]:
# Train the model
model.fit(
    X_train_scaled,
     {'department_output':y_train_encoded, 'attrition_output':y_train_encoded2},
    epochs = 100, batch_size = 35, validation_split = 0.2
    )


Epoch 1/100
26/26 [==============================] - 3s 21ms/step - loss: 1.3624 - department_output_loss: 0.8087 - attrition_output_loss: 0.5538 - department_output_accuracy: 0.6583 - attrition_output_accuracy: 0.7537 - val_loss: 1.2610 - val_department_output_loss: 0.8108 - val_attrition_output_loss: 0.4502 - val_department_output_accuracy: 0.6652 - val_attrition_output_accuracy: 0.8462
Epoch 2/100
26/26 [==============================] - 0s 5ms/step - loss: 1.2019 - department_output_loss: 0.7476 - attrition_output_loss: 0.4544 - department_output_accuracy: 0.6617 - attrition_output_accuracy: 0.8297 - val_loss: 1.2354 - val_department_output_loss: 0.8158 - val_attrition_output_loss: 0.4196 - val_department_output_accuracy: 0.6561 - val_attrition_output_accuracy: 0.8462
Epoch 3/100
26/26 [==============================] - 0s 5ms/step - loss: 1.1542 - department_output_loss: 0.7243 - attrition_output_loss: 0.4299 - department_output_accuracy: 0.6549 - attrition_output_accuracy: 0.8297

In [48]:
# Evaluate the model with the testing data
results = model.evaluate(X_test_scaled, {'department_output':y_test_encoded, 'attrition_output':y_test_encoded2})
results

12/12 [==============================] - 0s 9ms/step - loss: 6.3786 - department_output_loss: 4.5790 - attrition_output_loss: 1.7996 - department_output_accuracy: 0.4783 - attrition_output_accuracy: 0.7799


[6.3786163330078125,
 4.579039096832275,
 1.7995761632919312,
 0.47826087474823,
 0.779891312122345]

In [49]:
# Print the accuracy for both department and attrition
print(f'Department predictions accuracy: {results[3]}')
print(f'Attrition predictions accuracy: {results[4]}')

Department predictions accuracy: 0.47826087474823
Attrition predictions accuracy: 0.779891312122345


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. No, it is not the best metric to use, because some categories, especially for the department (HR), had very low counts compared to others.
2. I chose softmax for department because it is more categorical than binary, and chose sigmoid for attrition because it was binary.
3. When doing OHE, I think HR can be dropped because it was very low (in 90s) comapred to other two categories, which had high 3-digit numbers.